In [ ]:

import pandas as pd
import numpy as np
import sqlalchemy as sql
from decouple import config
import locale
import seaborn as sns
import matplotlib.pyplot as plt
import tempfile

pd.options.display.float_format = lambda x: '{:,.2f}'.format(x)

# Establecemos el locale para usar el formato español 
# (o de otros países que usan punto para miles y coma para decimales)
# locale.setlocale(locale=locale.LC_ALL, category='es_ES')

engine: sql.Engine = sql.create_engine(f"mssql+pyodbc://{config('SERVIDOR')}/Liquidacion?"
                           f"driver=ODBC+Driver+17+for+SQL+Server")
engine_plus: sql.Engine = sql.create_engine(f"mssql+pyodbc://{config('SERVIDOR_PLUS')}/" 
                        f"Liquidacion?driver=ODBC+Driver+17+for+SQL+Server")
engine_refuerzo: sql.Engine = sql.create_engine(f"mssql+pyodbc://"
    f"{config("SERVIDOR_REFUERZO")}/Liquidacion?driver="
    f"ODBC+Driver+17+for+SQL+Server""")

liquidacion  = config("LIQUIDACION")

meses_liq: list[str] = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 
    'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre',
    'Aguinaldo Junio', 'Aguinaldo Diciembre']

mes_liquidacion = int(liquidacion[-2:])
anio_liquidacion  = liquidacion[:4]
titulo_liquidacion  = meses_liq[mes_liquidacion - 1] + " de " + anio_liquidacion



In [ ]:
from pandas import DataFrame


cons_creditos = f"""SELECT L.PLLA, L.ORDEN, L.AFILIADO, SUM(MONTO) AS CREDITOS
FROM [{liquidacion}].dbo.Liquidacion L
INNER JOIN [{liquidacion}].dbo.Codigos C ON C.CODIGO=L.CODIGO
INNER JOIN Liquidacion.dbo.Codigos C2 ON C2.CODIGO=L.CODIGO
WHERE C.TIPO='C' AND C.CODIGO NOT IN ('05','07','0Y','00','01','0B','1Z','3Z','5Z','BC')
GROUP BY L.PLLA, L.ORDEN, L.AFILIADO""".format(liquidacion)

creditos: DataFrame = pd.read_sql(cons_creditos, engine)
cons_beneficios: str = f"""SELECT DL.CONTROL, DL.PLLA, DL.ORDEN, DL.AFILIADO, D.SEXO,
    DL.BENEFICIO, B.DETALLE AS BENEFICIO_TIPO, B.TIPO,
    R.DETALLE AS REPARTICION, R.TIPOREPARTICION, L.LOCALIDAD, FNAC 
    FROM DATOSAFILIADO D  
    LEFT OUTER JOIN BENEFICIO B ON B.COD=D.COD_BENEFICIO
    LEFT OUTER JOIN REPARTICION R ON R.COD=D.LETRA
    LEFT OUTER JOIN LOCALIDADES L ON D.CODLOCALIDAD=L.CODLOCALIDAD
    INNER JOIN [{liquidacion}].dbo.DatosAfiliado DL on DL.CONTROL=D.CONTROL""".format(
    liquidacion=liquidacion
)

beneficios: DataFrame = pd.read_sql(cons_beneficios, engine)
brutos: DataFrame = beneficios.merge(
    creditos, on=["PLLA", "ORDEN", "AFILIADO"], how="left"
).fillna(0.0)

brutos["BRUTO"] = brutos["BENEFICIO"] + brutos["CREDITOS"]

In [ ]:
# TIPOS REPARTICIONES - NORMALIZACION
brutos["TIPOREPARTICION"] = brutos["TIPOREPARTICION"].replace({
    "AGUAS": "Aguas",
    "NO": "Org.Desc.",
    "MG": "Adm. Central",
    "SP": "Salud Pública",
    "AC": "Adm. Central",
    "Legisladores": "P. Legislativo",
    "DOC": "Docente",
    "JUPOLICIA": "Policia",
    "MOVILIDADES": "Otras leyes",
    "Cautelares": "Otras leyes",
    "Certificado": "Otras leyes",
    "Especial": "Otras leyes",
})

In [ ]:
brutos["TIPO"] = brutos.apply(
    lambda x: "Retiros"
    if x["REPARTICION"] in ("Retiro Policial", "Retiro Carcelario")
    else "Pensiones"
    if x["TIPO"] == "pen"
    else "Jubilaciones",
    axis=1,
)

In [ ]:
# CONVERTIR LA COLUMNA FNAC DE BRUTOS A FECHA

brutos["FNAC"] = pd.to_datetime(
    brutos["FNAC"], format="%d/%m/%Y", errors="coerce"
)
brutos["EDAD"] = pd.to_datetime("today") - brutos["FNAC"]
brutos["EDAD"] = brutos["EDAD"].dt.days / 365
brutos["EDAD"] = brutos["EDAD"].astype(int, errors="ignore")

In [ ]:
# SANITIZO LOCALIDADES
brutos["LOCALIDAD"] = brutos["LOCALIDAD"].apply(
    lambda x: "SIN LOCALIDAD" if x == 0 else x
)
brutos["LOCALIDAD"] = brutos["LOCALIDAD"].apply(
    lambda x: "SAN LUIS\nDEL PALMAR" if x == "SAN LUIS DEL PALMAR" else x
)

In [ ]:
import pandas as pd

# Define el rango de brutos
inicio = 0
fin = 10000000
paso_1 = 100000
paso_2 = 1000000

# Crea las listas de inicio y fin
inicio_list = []
fin_list = []

# Agrega valores a las listas
inicio_list.append(0)
fin_list.append(250000)

for i in range(250000, 1000000, paso_1):
    inicio_list.append(i)
    fin_list.append(i + paso_1)

for i in range(1000000, fin, paso_2):
    inicio_list.append(i)
    fin_list.append(i + paso_2)

# Elimina el último elemento de fin_list si es necesario
if len(fin_list) > len(inicio_list):
    fin_list.pop()

# Crea un dataframe con el rango de brutos
rango_brutos = pd.DataFrame({
    'INICIO': inicio_list,
    'FIN': fin_list
})

print(rango_brutos)

In [ ]:
# cruzar brutos con rango_brutos y determinar cantidades en cada grupo

brutos.groupby(pd.cut(brutos["BRUTO"], rango_brutos["INICIO"])).size().to_clipboard()

In [ ]:
brutos.to_clipboard()

In [ ]:
""" CLASIFICAR BRUTOS POR rango_brutos, EN UNA NUEVA COLUMNA "RANGO" """

brutos["RANGO"] = pd.cut(brutos["BRUTO"], bins=rango_brutos["INICIO"], 
                         include_lowest=True, right=False)

In [ ]:
brutos.to_clipboard()